##### Cargar las librerías necesarias

In [24]:
!pip install pandas numpy matplotlib seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import kagglehub
!pip install kaggle
!pip install kaggle kaggle-cli
!pip install lxml
!pip install kagglehub



c:\Users\57322\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


  Using cached kaggle_cli-0.12.13-py3-none-any.whl
  Using cached cliff-2.8.3-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached MechanicalSoup-0.8.0-py2.py3-none-any.whl.metadata (795 bytes)
  Using cached lxml-4.0.0.tar.gz (4.2 MB)
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached cssselect-1.0.3-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached configparser-7.1.0-py3-none-any.whl.metadata (5.4 kB)
  Using cached progressbar2-3.34.3-py2.py3-none-any.whl.metadata (10 kB)
  Using cached beautifulsoup4-4.6.3-py3-none-any.whl.metadata (3.0 kB)
  Using cached pbr-6.1.0-py2.py3-none-any.whl.metadata (3.4 kB)
  Using cached prettytable-0.7.2-py3

  error: subprocess-exited-with-error
  
  × Building wheel for lxml (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [75 lines of output]
      Building lxml version 4.0.0.
      <string>:61: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
      Building without Cython.
      ERROR: b'"xslt-config" no se reconoce como un comando interno o externo,\r\nprograma o archivo por lotes ejecutable.\r\n'
      ** make sure the development packages of libxml2 and libxslt are installed **
      
      Using build configuration of libxslt
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-312\lxml
      copying src\lxml\builder.py -> build\lib.win-amd64-cpython-312\lxml
      copying src\lxml\cssselect.py -> build\lib.win-amd64-cpython-312\lxml
      copying src\lxml\doctestcompare.py -> build\lib.win-amd64-cpython-312\lxml
      copying src\lxml\Ele

##### Observar las bases de datos

In [ ]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("snehangsude/audible-dataset")
print("Path to dataset files:", path)

import pandas as pd
import os

# Ruta del dataset descargado
dataset_path = r"C:\Users\57322\.cache\kagglehub\datasets\snehangsude\audible-dataset\versions\1"

# Listar los archivos dentro del directorio
files = os.listdir(dataset_path)
print("Archivos en el directorio:", files)


Path to dataset files: C:\Users\57322\.cache\kagglehub\datasets\snehangsude\audible-dataset\versions\1
Archivos en el directorio: ['audible_cleaned.csv', 'audible_uncleaned.csv']


In [54]:

# Cargar el dataset original (sin limpiar)
original_file_path = os.path.join(dataset_path, "audible_uncleaned.csv")
df = pd.read_csv(original_file_path)

# Cargar el dataset limpio en cleaned_df
cleaned_file_path = os.path.join(dataset_path, "audible_cleaned.csv")
cleaned_df = pd.read_csv(cleaned_file_path)



In [55]:
# Mostrar las primeras filas de ambos datasets
print("\nDataset sin limpiar (df):")
print(df.head())

print("\nDataset limpio (cleaned_df):")
print(cleaned_df.head())


Dataset sin limpiar (df):
                                         name                     author  \
0                  Geronimo Stilton #11 & #12  Writtenby:GeronimoStilton   
1                            The Burning Maze      Writtenby:RickRiordan   
2                                The Deep End       Writtenby:JeffKinney   
3                        Daughter of the Deep      Writtenby:RickRiordan   
4  The Lightning Thief: Percy Jackson, Book 1      Writtenby:RickRiordan   

                    narrator                time releasedate language  \
0      Narratedby:BillLobely   2 hrs and 20 mins    04-08-08  English   
1   Narratedby:RobbieDaymond   13 hrs and 8 mins    01-05-18  English   
2      Narratedby:DanRussell    2 hrs and 3 mins    06-11-20  English   
3  Narratedby:SoneelaNankani  11 hrs and 16 mins    05-10-21  English   
4  Narratedby:JesseBernstein              10 hrs    13-01-10  English   

                           stars   price  
0     5 out of 5 stars34 ratings  

##### 1)  Cambiar el formato de hora y minuto hacia minutos totales

In [60]:
def convert_time_to_minutes(time_str):
    # Si el valor es un entero,  se devuelve  tal cual.
    if isinstance(time_str, int):  # Si time_str es un número entero
        return time_str  # Devuelve directamente el valor, ya que no requiere conversión.
    
    # Si el valor es NaN, 
    if pd.isnull(time_str):  # Si time_str es NaN (Not a Number), es decir, no tiene valor
        return 0  # Devuelve 0, ya que no hay tiempo que convertir.
    
    # Asegurarse de que 'time_str' sea una cadena
    time_str = str(time_str)  # Convierte el valor de time_str a string para garantizar que sea procesable.

    hours = 0  # Inicializa las horas en 0.
    minutes = 0  # Inicializa los minutos en 0.

    # Eliminamos el "and" que puede estar presente y luego dividimos la cadena por los espacios.
    parts = time_str.replace('and', '').split() 
    
    for i, part in enumerate(parts):  # Recorre cada parte de la lista de partes.
        if 'hr' in part:  # Si la parte contiene 'hr', es una indicación de horas.
            # Si hay un número antes de 'hr', se asigna a hours; si no, se asigna 0.
            hours = int(parts[i - 1]) if i > 0 else 0  
        elif 'min' in part:  # Si la parte contiene 'min', es una indicación de minutos.
            # Si hay un número antes de 'min', es a minutes; si no, se asigna 0.
            minutes = int(parts[i - 1]) if i > 0 else 0  

    # Devuelve el total de minutos (horas convertidas a minutos + minutos).
    return hours * 60 + minutes  # Calcula y devuelve el total de minutos: horas * 60 + minutos.

# Limpiar la columna 'time' convirtiendo a minutos
df['time'] = df['time'].apply(convert_time_to_minutes)  # Aplicar la función a toda la columna 'time' del DataFrame

# Mostrar las primeras filas de la columna 'time' para verificar la conversión
print(df['time'].head())  # Observar las primeras 5 filas para verificar cómo quedó la conversión.



0    140
1    788
2    123
3    676
4    600
Name: time, dtype: int64


##### 2) Al principio de la columnas Autor y Narrador, se elimina el prefijo "Writtenby" y "Narratedby", de forma correspondiente

In [76]:
# Limpiar la columna 'author' eliminando la cadena "Writtenby:"
df['author'] = df['author'].str.replace('Writtenby:', '', regex=False)

# Limpiar la columna 'narrator' eliminando la cadena "Narratedby:"
df['narrator'] = df['narrator'].str.replace('Narratedby:', '', regex=False)
print(df[['author', 'narrator']].head())

             author         narrator
0  Geronimo Stilton      Bill Lobely
1      Rick Riordan   Robbie Daymond
2       Jeff Kinney      Dan Russell
3      Rick Riordan  Soneela Nankani
4      Rick Riordan  Jesse Bernstein


##### 3. Extraer el número de estrellas y el número de valoraciones de la columna stars

In [65]:
# Asegurarse de que la columna 'stars' esté en formato de cadena
df['stars'] = df['stars'].astype(str)

# Separar la columna 'stars' en partes
stars_and_ratings = df['stars'].str.split(' ', expand=True)

# Verificar la longitud de las filas
print(stars_and_ratings.head())

# Comprobar si hay filas con menos de 5 partes y manejarlas
df['stars'] = stars_and_ratings[0]  # La primera parte es 'stars'
# Usamos `.get()` para evitar el KeyError si no hay suficientes partes
df['ratings'] = stars_and_ratings.get(4, pd.NA)  # Si no hay quinta parte, asignamos pd.NA

# Limpiar la columna 'ratings' para extraer solo los números
df['ratings'] = df['ratings'].str.extract('(\d+)').astype(float)

# Limpiar la columna 'stars' reemplazando 'Not' por 0 y convirtiéndola a float
df['stars'] = df['stars'].replace('Not', 0).astype(float)

# Ver las primeras filas para verificar los cambios
print(df[['stars', 'ratings']].head())




<>:16: SyntaxWarning: invalid escape sequence '\d'
<>:16: SyntaxWarning: invalid escape sequence '\d'
C:\Users\57322\AppData\Local\Temp\ipykernel_8488\398096750.py:16: SyntaxWarning: invalid escape sequence '\d'
  df['ratings'] = df['ratings'].str.extract('(\d+)').astype(float)


     0
0  5.0
1  4.5
2  4.5
3  4.5
4  4.5
   stars  ratings
0    5.0      NaN
1    4.5      NaN
2    4.5      NaN
3    4.5      NaN
4    4.5      NaN


##### 4. Separar los nombres y apellidos, así como asignarle mayúsculas a las columnas los autores y los narradores

In [78]:

# Agregar un espacio entre las letras minúsculas y mayúsculas en las columnas 'author' y 'narrator'
df['author'] = df['author'].str.replace(r'([a-z])([A-Z])', r'\1 \2', regex=True)
df['narrator'] = df['narrator'].str.replace(r'([a-z])([A-Z])', r'\1 \2', regex=True)

# Mostrar los primeros valores de las columnas
print(df['author'].head())
print(df[ "narrator"].head())



0    Geronimo Stilton
1        Rick Riordan
2         Jeff Kinney
3        Rick Riordan
4        Rick Riordan
Name: author, dtype: object
0        Bill Lobely
1     Robbie Daymond
2        Dan Russell
3    Soneela Nankani
4    Jesse Bernstein
Name: narrator, dtype: object


##### 5. Alteraciones a la columna price

In [ ]:
# Convertir a string
df['price'] = df['price'].astype(str)

# Reemplazar 'Free' por '0' para representar los productos gratuitos como cero
df['price'] = df['price'].replace('Free', '0')

# Eliminar las comas de los valores en la columna 'price'
df['price'] = df['price'].str.replace(',', '')

# Convertir la columna 'price' a tipo float para análisis numérico
df['price'] = df['price'].astype(float)

# Verificar los primeros valores de la columna 'price' después de la limpieza
print(df['price'])

0        468.0
1        820.0
2        410.0
3        615.0
4        820.0
         ...  
87484    596.0
87485    820.0
87486    938.0
87487    680.0
87488    569.0
Name: price, Length: 87489, dtype: float64
